In [1]:
import pandas as pd
df = pd.read_csv('corpus/speaknow_lyrics.csv', encoding = 'latin-1')

In [2]:
df.head()

,ï»¿album_name,track_title,track_n,lyric,line
0,Speak Now (Deluxe),Mine,1,"You were in college, working part-time, waitin...",1
1,Speak Now (Deluxe),Mine,1,Left a small town and never looked back,2
2,Speak Now (Deluxe),Mine,1,"I was a flight risk, with a fear of falling",3
3,Speak Now (Deluxe),Mine,1,"Wondering why we bother with love, if it never...",4
4,Speak Now (Deluxe),Mine,1,"I say, ""Can you believe it?""",5


In [3]:
songs = df.groupby('track_title').agg({'lyric': lambda x: " ".join(x)}).reset_index()

In [4]:
pd.options.display.max_colwidth = 5000

In [5]:
songs.head()

,track_title,lyric
0,Back to December,"I'm so glad you made time to see me How's life? Tell me, how's your family? I haven't seen them in a while You've been good, busier than ever We small talk, work and the weather Your guard is up and I know why Because the last time you saw me Is still burned in the back of your mind You gave me roses, and I left them there to die So this is me swallowing my pride Standing in front of you, saying I'm sorry for that night And I go back to December all the time It turns out freedom ain't nothing but missing you Wishing I'd realized what I had when you were mine I'd go back to December, turn around and make it alright I go back to December all the time These days, I haven't been sleeping Staying up, playing back myself leaving When your birthday passed and I didn't call And I think about summer, all the beautiful times I watched you laughing from the passenger side and Realized I loved you in the fall And then the cold came The dark days when fear crept into my mind You gave me all your love, and all I gave you was goodbye So this is me swallowing my pride Standing in front of you, saying I'm sorry for that night And I go back to December all the time It turns out freedom ain't nothing but missing you Wishing I'd realized what I had when you were mine I'd go back to December, turn around and change my own mind I go back to December all the time I miss your tan skin, your sweet smile So good to me, so right And how you held me in your arms that September night The first time you ever saw me cry Maybe this is wishful thinking Probably mindless dreaming But if we loved again, I swear I'd love you right I'd go back in time and change it, but I can't So if the chain is on your door, I understand But this is me swallowing my pride Standing in front of you, saying I'm sorry for that night And I go back to December It turns out freedom ain't nothing but missing you Wishing I'd realized what I had when you were mine I'd go back to December, turn around and make it alright I'd go back to December, turn around and change my own mind I go back to December all the time All the time"
1,Better Than Revenge,"Now go stand in the corner and think about what you did Ha, time for a little revenge The story starts when it was hot and it was summer and I had it all, I had him right there where I wanted him She came along, got him alone and let's hear the applause She took him faster than you can say ""Sabotage"" I never saw it coming, wouldn't have suspected it I underestimated just who I was dealing with She had to know the pain was beating on me like a drum She underestimated just who she was stealing from She's not a saint and she's not what you think She's an actress, whoa She's better known for the things that she does On the mattress, whoa Soon she's gonna find stealing other people's toys On the playground won't make you many friends She should keep in mind, she should keep in mind There is nothing I do better than revenge, ha She looks at life like it's a party and she's on the list She looks at me like I'm a trend and she's so over it I think her ever-present frown is a little troubling And she thinks I'm psycho 'cause I like to rhyme her name with things But sophistication isn't what you wear or who you know Or pushing people down to get you where you wanna go Oh, they didn't teach you that in prep school so it's up to me But no amount of vintage dresses gives you dignity (Think about what you did) Joeboy 'Sip (Alcohol)' Official Lyrics & Meaning | Verified She's not a saint and she's not what you think She's an actress, whoa She's better known for the things that she does On the mattress, whoa Soon she's gonna find stealing other people's toys On the playground won't make you many friends She should keep in mind, she should keep in mind There is nothing I do better than revenge, ha I'm just another thing for you to roll your eyes at, honey You might have him, but haven't you heard? I'm just another thin

In [6]:
len(songs)

17

In [7]:
import sklearn

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/anita/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
stop_words = stopwords.words('english')

In [11]:
stopwords = ["love", "mine", "man", "best", "hard", "little", 
    "come", "back", "gone", "thought", "oh", "figured", 
    "leave", "could", "like", "go", "away", "la", "meet", 
    "say", "sparks", "go", "watch", "kiss", "something", 
    "think", "better", "story", "whoa", "know", "many", 
    "known", "us", "mean", "ever", "big", "someday", 
    "gonna", "old", "city", "oh", "little", "never", 
    "still", "long", "lights", "oh", "alright", "time", 
    "life", "wait", "days"]

In [12]:
vectorizer = TfidfVectorizer(stop_words = stop_words, min_df = 0.1)

In [13]:
tfidf = vectorizer.fit_transform(songs['lyric'])

In [14]:
from sklearn.decomposition import NMF

In [15]:
nmf = NMF(n_components = 6)

In [16]:
topic_values = nmf.fit_transform(tfidf)

In [17]:
for topic_num, topic in enumerate(nmf.components_):
    message = "Topic #{}: ".format(topic_num + 1)
    message += " ".join([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11 :-1]])
    print(message)
    

Topic #1: love water mine time shine man people best hard little
Topic #2: come back gone thought oh figured leave could like go
Topic #3: fly away la meet say sparks go watch kiss something
Topic #4: think better story whoa know like many known us oh
Topic #5: mean ever big someday gonna old city oh little never
Topic #6: still long live lights oh alright time life wait days


In [18]:
topic_labels = ['nature', 'return and departure', 'adventure', 'reflection', 'future', 'persistence']

In [19]:
df_topics = pd.DataFrame(topic_values, columns = topic_labels)
df_topics

,nature,return and departure,adventure,reflection,future,persistence
0,0.159517,0.280810,0.015215,0.000000,0.000000,0.223003
1,0.003563,0.039556,0.000000,0.785156,0.000000,0.000000
2,0.000000,0.153375,0.000000,0.464151,0.000000,0.141474
3,0.114252,0.000000,0.230300,0.017295,0.006390,0.112612
4,0.000000,0.732736,0.000000,0.031178,0.063815,0.000000
5,0.000000,0.800387,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.811462
7,0.000000,0.000000,0.355640,0.125777,0.000000,0.125832
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.722911
9,0.000000,0.000000,0.000000,0.000000,0.840410,0.000000


In [20]:
songs = songs.join(df_topics)

In [21]:
#topic is in
songs.loc[songs['nature'] >= 0.1, 'nature'] = 1
songs.loc[songs['return and departure'] >= 0.1, 'return and departure'] = 1
songs.loc[songs['adventure'] >= 0.1, 'adventure'] = 1
songs.loc[songs['reflection'] >= 0.1, 'reflection'] = 1
songs.loc[songs['future'] >= 0.1, 'future'] = 1
songs.loc[songs['persistence'] >= 0.1, 'persistence'] = 1

In [22]:
#topic is in
songs.loc[songs['nature'] < 0.1, 'nature'] = 0
songs.loc[songs['return and departure'] < 0.1, 'return and departure'] = 0
songs.loc[songs['adventure'] < 0.1, 'adventure'] = 0
songs.loc[songs['reflection'] < 0.1, 'reflection'] = 0
songs.loc[songs['future'] < 0.1, 'future'] = 0
songs.loc[songs['persistence'] < 0.1, 'persistence'] = 0

In [23]:
songs_csv = songs.to_csv('topics/speaknow_topics.csv')